In [2]:
import requests, re
from bs4 import BeautifulSoup

In [ ]:
r=requests.get("https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
c=r.content

soup=BeautifulSoup(c,"html.parser")

Necesito conocer la estructura de la web, por lo que lo adecuado es inspeccionarla y a partir de ahí seguir extrayendo datos

In [14]:
all = soup.find_all("div", {"class": "propertyRow"})
print(len(all)) # Debe ser 10, que son los resultados que hay en una página

all[0].find_all

10


<bound method Tag.find_all of <div class="propertyRow" id="propertyRowREN021201395" onclick="Track.doEvent('Hybrid Mapping', 'Property Center Lane', 'Select a property with brand REN to view details'); document.location.href='/property/0-gateway-rock-springs-wy-82901-REN021201395';">
<div class="CenterLaneCardBg CardWrapper propertyCard" id="propertyREN021201395">
<div class="CenterLaneCard propertyCard">
<div class="CardThumb">
<div class="landscapeThumbContainer">
<a href="http://web.archive.org/web/20160127020422/http://www.century21.com/property/0-gateway-rock-springs-wy-82901-REN021201395"><img class="lazyLoad" rel="http://www.century21.com/listhub/thumb/130x90/photos.listhub.net/FCBRWY/20156419/1?lm=20151103T230938" src="./LCWYROCKSPRINGS1_files/1"/></a>
</div>
</div>
<h4 class="propPrice">


            $725,000



      <span class="IconPropertyFavorite16"></span>
</h4>
<div class="CardDetails">
<div class="primaryDetails">
<span class="propAddressCollapse" title="0 Gateway">0 

In [8]:
all[0].find_all("h4", {"class": "propPrice"})

[<h4 class="propPrice">
 
 
             $725,000
 
 
 
       <span class="IconPropertyFavorite16"></span>
 </h4>]

In [10]:
all[0].find("h4", {"class": "propPrice"}).text

'\n\n\n            $725,000\n\n\n\n      \n'

In [ ]:
all[0].find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", "") 
# Reemplazo los \n y los espacios en blanco para que quede solo el precio

'$725,000'

In [15]:
for item in all:
    print(item.find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", ""))

$725,000
$452,900
$396,900
$389,900
$254,000
$252,900
$210,000
$209,000
$199,900
$196,900
